In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
#Loading data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gend = pd.read_csv('gender_submission.csv')

print(train.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [3]:
median_age = train["Age"].median()
train['Age'] = train['Age'].fillna(median_age)

In [4]:
train = train.drop(columns = ["Cabin"])
test = test.drop(columns = ["Cabin"])

In [5]:
print(train['Embarked'].isnull().sum())
print(train['Embarked'].value_counts())

2
Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


In [6]:
train['Embarked'] = train['Embarked'].fillna('S')
print(train['Embarked'].value_counts())

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64


In [7]:
test.isnull().sum()
test['Age'] = test['Age'].fillna(median_age)


In [8]:
mean_fare = train['Fare'].mean()
test['Fare'] = test['Fare'].fillna(mean_fare)



In [9]:
train_encoded = pd.get_dummies(train, columns=['Sex'], drop_first=True, dtype=int)
test_encoded = pd.get_dummies(test, columns=['Sex'], drop_first=True, dtype=int)

In [10]:
print(train_encoded['Embarked'].unique())
print(test_encoded['Embarked'].unique())

['S' 'C' 'Q']
['Q' 'S' 'C']


In [11]:
train_encoded1 = pd.get_dummies(train_encoded, columns=['Embarked'] ,drop_first=True, dtype=int)
test_encoded1 = pd.get_dummies(test_encoded, columns=['Embarked'] ,drop_first=True, dtype=int)
train_encoded1 = train_encoded1.drop(columns=["Ticket"])
test_encoded1 = test_encoded1.drop(columns=["Ticket"])

In [12]:
scaler = StandardScaler()

In [13]:
print(train_encoded1.describe())
# print(test_encoded1.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.361582    0.523008   
std     257.353842    0.486592    0.836071   13.019697    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   22.000000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   35.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare    Sex_male  Embarked_Q  Embarked_S  
count  891.000000  891.000000  891.000000  891.000000  891.000000  
mean     0.381594   32.204208    0.647587    0.086420    0.725028  
std      0.806057   49.693429    0.477990    0.281141    0.446751  
min      0.000000    0.000000    0.000000    0.000000    0.000000  
25%      0.000000    7.910400

In [14]:
train_age = train_encoded1[['Age']]
test_age = test_encoded1[['Age']]

In [15]:
agetrain_encoded = scaler.fit_transform(train_age)
agetest_encoded = scaler.transform(test_age)

In [16]:
train_age_scaled = pd.DataFrame(agetrain_encoded, columns=["Age"], index=train_encoded1.index)
test_age_scaled = pd.DataFrame(agetest_encoded, columns=["Age"], index=test_encoded1.index)

In [17]:
train_encoded1['Age'] = train_age_scaled
test_encoded1['Age'] = test_age_scaled

In [18]:
train_encoded1[['Fare']] = scaler.fit_transform(train_encoded1[['Fare']])
test_encoded1[['Fare']] = scaler.transform(test_encoded1[['Fare']])

In [19]:
train_encoded1 = train_encoded1.drop(columns='Name')
test_encoded1 = test_encoded1.drop(columns='Name')

In [20]:
X = train_encoded1.drop(columns=['Survived'])
y = train_encoded1['Survived']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [22]:
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

LogisticRegression(max_iter=500)

In [23]:
model_tree = DecisionTreeClassifier(random_state=2)
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=2)

In [24]:
#Prediction
y_pred_tree = model_tree.predict(X_test)
y_val_prediction = model.predict(X_test) 

In [27]:
accuracy = accuracy_score(y_test, y_val_prediction)
accuracy_tre = accuracy_score(y_test, y_pred_tree)
print(f"{accuracy:.3f} Logistic Regression")
print(f"{accuracy_tre:.3f} Decision Tree")

0.804 Logistic Regression
0.799 Decision Tree
